In [101]:
from pathlib import Path
import os
import pandas as pd
Path.cont = lambda x: list(os.scandir(x)) # scandir gives all content of a dir 
filedir = 'RIVM_data'
files = Path.cont(filedir)

def generate_df(filedir,filename,i):
    df = pd.read_csv(filedir+'//'+filename.name,encoding = 'cp1252',skiprows=1)
    df['BevAant'] = pd.to_numeric(df['BevAant'])
    df['Aantal'] = pd.to_numeric(df['Aantal'])
    df.rename(columns={'Aantal per 100.000 inwoners':'Aantal per 100000 inwoners'},inplace=True)
    df['Datum'] = filename.name[:10]
    df['Count'] = str(i)
    print(filename.name)
    return df
df = pd.concat([generate_df(filedir,filename,i) for i,filename in enumerate(files)])
num_csv = len(files)
df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_aantal.head()

03-19-2020 13-52.csv
03-20-2020 09-29.csv
03-20-2020 19-28.csv


Count,Gemeente,0,1,2
0,NaN,NaN,NaN,NaN
1,'s-Gravenhage,21.0,27.0,36.0
2,'s-Hertogenbosch,30.0,35.0,38.0
3,* Bij 112 personen is de woonplaats niet van b...,NaN,NaN,99.0
4,* postcode ontbreekt bij 84 pt en 1 woont in b...,99.0,NaN,NaN


In [102]:
df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_aantal.head()

Count,Gemeente,0,1,2
0,NaN,NaN,NaN,NaN
1,'s-Gravenhage,21.0,27.0,36.0
2,'s-Hertogenbosch,30.0,35.0,38.0
3,* Bij 112 personen is de woonplaats niet van b...,NaN,NaN,99.0
4,* postcode ontbreekt bij 84 pt en 1 woont in b...,99.0,NaN,NaN


In [103]:
# df.columns = ['_'.join(col) for col in df.columns.values]
# df

In [104]:
import altair as alt
from vega_datasets import data

url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 

gemeentes = alt.topo_feature(url2, 'gemeente_2020')

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
w = widgets.IntSlider()

columns = [str(day) for day in range(0, 30)]
slider = alt.binding_range(min=0, max=30, step=1)
select_day = alt.selection_single(name="day", fields=['day'],
                                   bind=slider, init={'day': 1})

def generate_chart(col):
  # print(col)

  chart = alt.Chart(gemeentes).mark_geoshape().properties(
    width=800,
    height=500
    ).transform_lookup(
    lookup='properties.statnaam',
    from_=alt.LookupData(df, 'Gemeente_', [col]),
    default= '0'
    ).encode(
    color=alt.Color(col+':Q',scale=alt.Scale(type='symlog')),
    tooltip=['properties.statnaam:N', col+':Q']
    )
  return chart

generate_chart('Aantal_1')|generate_chart('Aantal_9')


alt.HConcatChart(...)

In [105]:
import altair as alt
import pandas as pd
from vega_datasets import data
columns = [str(day) for day in range(0, num_csv)]

slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="day", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})
def serve_chart(df, legend):
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns)
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='redyellowgreen', type='symlog',domain=[200, 0])),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=350,
        height=400
    ).transform_filter(
        select_year
    )
    
    return chart
serve_chart(df_aantal, 'aantal') | serve_chart(df_relative, 'aantal')

alt.HConcatChart(...)

In [106]:
alt.hconcat(
    serve_chart(df_aantal, 'absoluut'), serve_chart(df_relative, 'relatief')
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.HConcatChart(...)

In [65]:
df_aantal[df_aantal['Gemeente']=='Uden']

Count,Gemeente,0,1,2,3
296,Uden,53.0,60.0,60.0,71.0


In [66]:
df_relative[df_relative['Gemeente']=='Uden']

Count,Gemeente,0,1,2,3
296,Uden,126.8,143.6,143.6,169.9
